# Classification
Classify malware samples according to their labels using their reduced representations.

In [1]:
import tensorflow as tf
print(f"GPU is {'not ' if len(tf.config.list_physical_devices('GPU')) == 0 else ''}available")

2024-04-29 15:46:38.386891: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 15:46:38.910280: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


GPU is available


2024-04-29 15:46:39.567518: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 15:46:39.605237: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 15:46:39.605282: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
import keras

# Data Processing

The `1-Data-Exploration.ipynb` file should have been run before this, and, as a result, the `processed-data.csv` file should have been generated.

In [3]:
import pandas as pd

In [4]:
raw_df = pd.read_csv("../data/processed-data.csv")
raw_df

,label,hash,dim-00,dim-01,dim-02,dim-03,dim-04,dim-05,dim-06,dim-07,...,dim-22,dim-23,dim-24,dim-25,dim-26,dim-27,dim-28,dim-29,dim-30,dim-31
0,TRICKBOT,f16631469eb35406ef4049d30c763cadda571b25bbdb45...,7.549576,0.000000,0.0,8.502933,24.176190,15.322744,6.537903,17.014332,...,3.867043,0.0,2.907426,0.000000,0.000000,14.940077,0.000000,0.0,0.0,0.000000
1,DARKKOMET,d31a7102cbc54447c251ba62760eb484fd0c9fbb8ea54f...,0.000000,0.000000,0.0,14.619308,0.000000,0.000000,16.847560,7.412828,...,12.765860,0.0,0.000000,9.257461,12.005179,13.770399,12.168733,0.0,0.0,8.010514
2,COBALTSTRIKE,dab956e9c864a84d12e8106a24ac3cf2950394152c62b6...,9.411519,11.802638,0.0,8.384830,0.000000,7.831763,9.012511,0.000000,...,0.000000,0.0,0.000000,7.769751,0.000000,4.577077,0.000000,0.0,0.0,8.902426
3,HIVE,122e397dc3a55143bd276d6ff3bc04a05601fbf390aa52...,16.296764,8.219131,0.0,4.276267,6.688067,6.247597,1.341092,5.731432,...,0.307458,0.0,11.367329,3.165555,0.000000,11.909040,0.000000,0.0,0.0,9.032462
4,AGENTB,b390756adcaa12f5abf6c97174d376027a85efd7606ee5...,20.252020,10.148681,0.0,7.180245,24.111082,8.394599,4.088972,9.327502,...,0.697547,0.0,13.447311,0.000000,0.000000,15.865129,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,FARFLI,de965e33dff58cf011106feacef2f804d9e35d00b8b5ff...,6.590584,1.287567,0.0,2.330490,0.000000,9.913089,2.610625,5.197479,...,4.686391,0.0,5.719362,5.128968,0.617545,7.388851,0.000000,0.0,0.0,8.240299
856,TRICKBOT,9506421d996290f70689559ee0c09cc074c948fff49547...,0.000000,0.000000,0.0,12.429248,0.000000,13.549780,15.505765,13.514245,...,20.763780,0.0,0.000000,0.000000,6.163751,0.922269,0.000000,0.0,0.0,1.915500
857,NECONYD,0eee965f286f057a3175797590795bbf99fda65dc8d845...,20.171080,4.978172,0.0,0.000000,3.767549,13.185392,17.561668,4.622786,...,0.373729,0.0,0.000000,6.819319,0.000000,5.469461,0.000000,0.0,0.0,0.000000
858,TRICKBOT,0c6aa0ae05d5fa8bf5a8ea95310be73ee60e55a0ce6864...,7.412106,3.795544,0.0,3.142282,0.000000,8.510063,9.629321,6.728159,...,16.329900,0.0,6.062054,8.711801,12.110841,13.719257,0.000000,0.0,0.0,10.587579


For training the model, we don't need the `hash` of the sample.

In [5]:
df = raw_df.drop(columns=["hash"])
df

,label,dim-00,dim-01,dim-02,dim-03,dim-04,dim-05,dim-06,dim-07,dim-08,...,dim-22,dim-23,dim-24,dim-25,dim-26,dim-27,dim-28,dim-29,dim-30,dim-31
0,TRICKBOT,7.549576,0.000000,0.0,8.502933,24.176190,15.322744,6.537903,17.014332,0.0,...,3.867043,0.0,2.907426,0.000000,0.000000,14.940077,0.000000,0.0,0.0,0.000000
1,DARKKOMET,0.000000,0.000000,0.0,14.619308,0.000000,0.000000,16.847560,7.412828,0.0,...,12.765860,0.0,0.000000,9.257461,12.005179,13.770399,12.168733,0.0,0.0,8.010514
2,COBALTSTRIKE,9.411519,11.802638,0.0,8.384830,0.000000,7.831763,9.012511,0.000000,0.0,...,0.000000,0.0,0.000000,7.769751,0.000000,4.577077,0.000000,0.0,0.0,8.902426
3,HIVE,16.296764,8.219131,0.0,4.276267,6.688067,6.247597,1.341092,5.731432,0.0,...,0.307458,0.0,11.367329,3.165555,0.000000,11.909040,0.000000,0.0,0.0,9.032462
4,AGENTB,20.252020,10.148681,0.0,7.180245,24.111082,8.394599,4.088972,9.327502,0.0,...,0.697547,0.0,13.447311,0.000000,0.000000,15.865129,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,FARFLI,6.590584,1.287567,0.0,2.330490,0.000000,9.913089,2.610625,5.197479,0.0,...,4.686391,0.0,5.719362,5.128968,0.617545,7.388851,0.000000,0.0,0.0,8.240299
856,TRICKBOT,0.000000,0.000000,0.0,12.429248,0.000000,13.549780,15.505765,13.514245,0.0,...,20.763780,0.0,0.000000,0.000000,6.163751,0.922269,0.000000,0.0,0.0,1.915500
857,NECONYD,20.171080,4.978172,0.0,0.000000,3.767549,13.185392,17.561668,4.622786,0.0,...,0.373729,0.0,0.000000,6.819319,0.000000,5.469461,0.000000,0.0,0.0,0.000000
858,TRICKBOT,7.412106,3.795544,0.0,3.142282,0.000000,8.510063,9.629321,6.728159,0.0,...,16.329900,0.0,6.062054,8.711801,12.110841,13.719257,0.000000,0.0,0.0,10.587579


Separate into `X` and `y`.

In [6]:
X = df.drop(columns=["label"])
y = df["label"]

In [7]:
print(X.shape)
print(y.shape)

(860, 32)
(860,)


# Label Encoding and Train-Test Split

We need to encode the labels for the `y`.

In [8]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

We want to save the label encoder for use in the full model.

In [9]:
import pickle

with open("../models/classifier/label-encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

80% of the data will be saved for training, while 20% will be left for testing.

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [11]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(688, 32)
(172, 32)
(688,)
(172,)


# Model Creation

Nothing too special here, we just use several dense layers for classification.

In [12]:
from keras import layers
from keras.models import Sequential

In [13]:
INITIAL_EPOCH = 0

def create_model():
    model = Sequential(name="Classifier")

    model.add(keras.Input((X.shape[1],), name="input"))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(256, activation="relu"))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(len(label_encoder.classes_), activation="softmax"))

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer="adam",
        metrics=["sparse_categorical_accuracy"]
    )
    return model

In [14]:
model = create_model()
model.summary()

2024-04-29 15:46:39.908215: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 15:46:39.908312: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 15:46:39.908346: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 15:46:40.814126: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 15:46:40.814257: I external/local_xla/xla/stream_executor

Model: "Classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 61)             │         3,965 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 39,165 (152.99 KB)

 Trainable params: 39,165 (152.99 KB)

 Non-trainable params: 0 (0.00 B)

Define callbacks.

In [15]:
import os

checkpoint_path = "../models/classifier/checkpoint.keras"
checkpoint_dir = os.path.dirname(checkpoint_path)

checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

In [16]:
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, verbose=1, min_delta=1e-4)

Load latest checkpoint if there is one.

In [17]:
# INITIAL_EPOCH = 54
# model = keras.models.load_model(checkpoint_path)

Train the model.

In [18]:
NUM_EPOCHS = 100

model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    initial_epoch=INITIAL_EPOCH,
    epochs=NUM_EPOCHS,
    callbacks=[checkpointer, early_stopping]
)

Epoch 1/100


I0000 00:00:1714376801.978351 1193293 service.cc:145] XLA service 0x7fd6a000cbf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714376801.978392 1193293 service.cc:153]   StreamExecutor device (0): Quadro P1000, Compute Capability 6.1
2024-04-29 15:46:41.996183: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-29 15:46:42.800216: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


 1/18 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/step - loss: 5.2456 - sparse_categorical_accuracy: 0.0000e+00

I0000 00:00:1714376804.043247 1193293 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 4.4550 - sparse_categorical_accuracy: 0.0632   
Epoch 1: val_loss improved from inf to 3.45563, saving model to ../models/classifier/checkpoint.keras
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 116ms/step - loss: 4.4347 - sparse_categorical_accuracy: 0.0667 - val_loss: 3.4556 - val_sparse_categorical_accuracy: 0.2754
Epoch 2/100
 1/18 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 3.2057 - sparse_categorical_accuracy: 0.2812
Epoch 2: val_loss improved from 3.45563 to 2.89029, saving model to ../models/classifier/checkpoint.keras
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.2114 - sparse_categorical_accuracy: 0.2668 - val_loss: 2.8903 - val_sparse_categorical_accuracy: 0.3333
Epoch 3/100
 1/18 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 3.0663 - sparse_categorical_accuracy: 0.2812
Epoch 3: val_loss improved from 2.89029 to 2.57410, saving model to ../models/classifier/checkpoint.keras
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.7298 - sparse_categoric

Load the best performing model.

In [19]:
model = keras.models.load_model(checkpoint_path)

In [20]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print(f"Testing loss:      {test_loss:5.5f}")
print(f"Testing accuracy:  {test_acc:5.5f}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - loss: 2.0054 - sparse_categorical_accuracy: 0.5606
Testing loss:      1.92156
Testing accuracy:  0.58721


In [21]:
model.save("../models/classifier/classifier.keras")